# Algorithms with Matrices

In this practice we are going to implement algorithms to work with matrices:
- solving simple equations
- inverting matrices
- decomposing matrices to LU decompositions

For these activities Gauss Elimination is our main tool. We will only work with matrices that do not require swapping the rows.

We will also perform QR decomposition using Gramm-Schmidt orthonormalization method

In [8]:
import numpy as np
import lovely_numpy as ln
import json_tricks 
import matplotlib.pyplot as plt

In [9]:
np.random.seed(0)


inputs = json_tricks.load('inputs/inputs.json')
answer = {}

In [10]:
print(len(inputs['nullspace_cases']))

30


# Task 1. Implement Gauss Elimination Stage I

## 1.1 Gauss Elimination -- non-pivoted case
Firstly, your task here is to create an algorithm that will perfrom mutations of matrix $A$ that transform it to Upper-Triangular form $U$.

$$
\begin{bmatrix}
a_{11} & a_{12} & \dots & a_{1N} \\
a_{21} & a_{22} & \dots & a_{2N} \\
\vdots & \vdots & \ddots & \vdots \\
a_{N1} & a_{N2} & \dots & a_{NN}
\end{bmatrix}

\rightarrow

\begin{bmatrix}
1 & a_{12}^* & \dots & a_{1N}^* \\
0 & 1 & \dots & a_{2N}^* \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \dots & 1
\end{bmatrix}
$$

Note that there should be unit diagonal after the 1 stage.

The same mutations for rows as you do to the matrix $A$ should be done to the rows of matrix $B$. 

What can matrix $B$ be? It can be:
- vector of right hand side $b$ (in that case we are solving the System of Linear Equations)
- identity matrix (in that case we invert matrix)
- actually, Stage I can be used as Stage II, and in that case right hand matrix can be some matrix from Stage I

## 1.2 Add pivoting

There is a flag `pivoting`. In case this flag is turned on, the Gauss Elimination should swap the current row with 
the row with maximal absolute value in the current column.

You should also perform the same swap with the matrix $B$ and also you should keep track of swaps with the matrix
$P$.

## 1.3 Account for the cases with null-space

In that case once in a while you will not be able to perform pivoting to evade the division-by-zero. In that case you
will find out that the diagonal element and all the elements below are all together less than some threshold value.

In that case just continue to the next column.

Keep in mind though that you have to advance only the column index, but the row index should not be increased. That
means that now your pivot is off the main diagonal.


In [11]:
def gauss_elimination(A, B, pivoting=False):
    A = A.copy().astype(float)
    B = B.copy().astype(float)

    n_rows, n_cols = A.shape
    P = np.eye(n_rows)  
    row = 0
    col = 0
    eps = 1e-12
    while row < n_rows and col < n_cols:
        if pivoting:
            max_row = np.argmax(np.abs(A[row:, col])) + row
            if np.abs(A[max_row, col]) > eps:
                if max_row != row:
                    A[[row, max_row], :] = A[[max_row, row], :]
                    B[[row, max_row], ...] = B[[max_row, row], ...]
                    P[[row, max_row], :] = P[[max_row, row], :]
            else:
                col += 1
                continue
        else:
            if np.abs(A[row, col]) < eps:
                if np.abs(A[row:, col]).max() < eps:
                    col += 1
                    continue
        pivot = A[row, col]
        if np.abs(pivot) > eps:
            A[row, :] = A[row, :] / pivot
            B[row, ...] = B[row, ...] / pivot
            for r in range(row + 1, n_rows):
                factor = A[r, col]
                A[r, :] -= factor * A[row, :]
                B[r, ...] -= factor * B[row, ...]
            row += 1
        col += 1
    return P, A, B

In [12]:
answer['gauss_elimination_simple_cases'] = [
    gauss_elimination(A=input['A'], B=input['b'], pivoting=False) for input in inputs['simple_cases']
]

answer['gauss_elimination_pivoting_cases'] = [
    gauss_elimination(A=input['A'], B=input['b'], pivoting=True) for input in inputs['simple_cases']
]

answer['gauss_elimination_nullspace_cases'] = [
    gauss_elimination(A=input['A'], B=input['b'], pivoting=True) for input in inputs['nullspace_cases']
]

# Task 2: Solve System of Linear Equations (SLE)

Now we have the engine to solve a lot of things. We will start with solving Systems of Linear Equations (SLEs).

For the SLE given by matrix $A$ and right-hand vector $\mathbf b$, implement a function that solves this SLE.

You will need to use twice Gauss Elimination Algorithm

1. Apply it to matrix $A$ and to vector $\mathbf b$ as matrix $B$ as Gauss Elimintaion I stage
2. For Gauss Elimination II stage you can use the same algorithm by just applying it to reversed matrix 

    (by enumerating equations and variables in reversed order, we can rewrite the matrix $U$ in low-triangle form $L$). 

    Something has to be done to the right-hand side vector too
3. Form the answer from output of Gauss Elimination Stage II

In [13]:
def solve_equation(A, b):
    n = A.shape[0]
    _, U, y = gauss_elimination(A, b.reshape(-1, 1), pivoting=True)
    _, _, x = gauss_elimination(U[::-1, ::-1], y[::-1, :], pivoting=False)
    return x[::-1, :].flatten()

You can always check that your solution is correct by subtracting

$A \mathbf x - \mathbf b$, which should give you $\mathbf 0$

By the way, matrix product, such as $A \mathbf x$ in numpy is done using `A @ x` operation

In [14]:
answer['solutions_simple_cases'] = [
    solve_equation(**input) for input in inputs['simple_cases']
]

answer['solution_pivoting_cases'] = [
    solve_equation(**input) for input in inputs['pivoting_cases']
]

# This should be close to zero
print("=== SIMPLE CASES ===")
for index in range(len(inputs['simple_cases'])):
    print(np.abs(inputs['simple_cases'][index]['A'] @ answer['solutions_simple_cases'][index] - inputs['simple_cases'][index]['b']).sum())

print("=== PIVOTING CASES ===")
for index in range(len(inputs['pivoting_cases'])):
    print(np.abs(inputs['pivoting_cases'][index]['A'] @ answer['solution_pivoting_cases'][index] - inputs['pivoting_cases'][index]['b']).sum())

=== SIMPLE CASES ===
1.7763568394002505e-14
4.263256414560601e-14
0.0
8.171241461241152e-14
3.552713678800501e-15
8.881784197001252e-16
1.7763568394002505e-14
0.0
1.438849039914203e-13
1.1013412404281553e-13
0.0
3.197442310920451e-14
1.0080825063596421e-13
3.907985046680551e-14
7.105427357601002e-15
4.440892098500626e-15
0.0
0.0
0.0
0.0
3.197442310920451e-14
0.0
0.0
1.5987211554602254e-14
0.0
5.329070518200751e-15
0.0
2.3092638912203256e-14
3.197442310920451e-14
0.0
=== PIVOTING CASES ===
7.105427357601002e-14
3.197442310920451e-14
7.993605777301127e-14
1.1324274851176597e-13
0.0
4.263256414560601e-14
3.019806626980426e-14
6.394884621840902e-14
1.4921397450962104e-13
1.2434497875801753e-13
7.105427357601002e-15
0.0
4.263256414560601e-14
1.4210854715202004e-14
7.105427357601002e-15
7.460698725481052e-14
2.8421709430404007e-13
0.0
1.509903313490213e-14
9.592326932761353e-14
1.0480505352461478e-13
1.8474111129762605e-13
2.2382096176443156e-13
0.0
0.0
1.4210854715202004e-14
2.4868995751603

# Task 3: Find inverse of $A$

Use the same approach as with solving SLE, but now instead of $\mathbf b$ for right hand side, insert corresponding Identity matrix $I$ as input into Gaussian Elimination algorithm

Identitiy matrix of size $N \times N$ is built using `numpy.eye(N)` 

In [15]:
def find_inverse(A):
    res = None
    ## YOUR CODE HERE
    # --placeholder start--
    P1, A1, B1 = gauss_elimination(A, np.eye(A.shape[0]), pivoting=True)
    P2, A2, B2 = gauss_elimination(A1[::-1, ::-1], B1[::-1, ::-1], pivoting=False)
    res = B2[::-1, ::-1]
    # --placeholder end--
    return res

You can always check that the matrix that you have found is indeed inverse to $A$ as $A^{-1} A = I$

In [16]:
answer['simple_inverses'] = [
    find_inverse(input['A']) for input in inputs['simple_cases']
]

answer['pivoting_inverses'] = [
    find_inverse(input['A']) for input in inputs['pivoting_cases']
]

# This should be I
print("=== SIMPLE CASES ===")
for index in range(len(inputs['simple_cases'])):
    print(answer['simple_inverses'][0] @ inputs['simple_cases'][0]['A'])

print("=== PIVOTING CASES ===")
for index in range(len(inputs['pivoting_cases'])):
    print(answer['pivoting_inverses'][0] @ inputs['pivoting_cases'][0]['A'])

=== SIMPLE CASES ===
[[ 1.00000000e+00  1.42108547e-14  2.84217094e-14  7.10542736e-15
  -1.77635684e-14  2.84217094e-14]
 [ 7.81597009e-14  1.00000000e+00 -1.13686838e-13 -2.84217094e-14
   6.39488462e-14 -1.13686838e-13]
 [-2.48689958e-14  2.84217094e-14  1.00000000e+00  1.42108547e-14
  -3.19744231e-14  5.68434189e-14]
 [-2.66453526e-15  3.55271368e-15  3.55271368e-15  1.00000000e+00
  -3.55271368e-15  7.10542736e-15]
 [ 5.32907052e-15 -5.32907052e-15 -1.24344979e-14 -7.10542736e-15
   1.00000000e+00 -7.10542736e-15]
 [-5.32907052e-15  5.32907052e-15  1.24344979e-14  0.00000000e+00
  -8.88178420e-15  1.00000000e+00]]
[[ 1.00000000e+00  1.42108547e-14  2.84217094e-14  7.10542736e-15
  -1.77635684e-14  2.84217094e-14]
 [ 7.81597009e-14  1.00000000e+00 -1.13686838e-13 -2.84217094e-14
   6.39488462e-14 -1.13686838e-13]
 [-2.48689958e-14  2.84217094e-14  1.00000000e+00  1.42108547e-14
  -3.19744231e-14  5.68434189e-14]
 [-2.66453526e-15  3.55271368e-15  3.55271368e-15  1.00000000e+00
  -

# Task 4: LU decomposition

You can find LU decomposition of the matrix $A$ using Gauss Elimination. Again, it is done using 2 stages of Gauss Elimination:

1. $A \mathbf x = I \mathbf b \rightarrow U \mathbf x = L^* \mathbf b$

2. $I U \mathbf x = L^* \mathbf b \rightarrow L U \mathbf x = I \mathbf b$

But in case you are doing pivoted version, it is a little more tricky. You have to modify somehow the matrix $L$
after the first stage of Gauss Elimination.

In [17]:
def find_plu(A, pivoting=True):
    A = A.copy().astype(float)
    n = A.shape[0]

    # Step 1: Get P, U, and L_inv using pivoted Gauss elimination
    # The third output is L_inv: inverse of lower triangular matrix L
    P, U, L_inv = gauss_elimination(A, np.eye(n), pivoting=pivoting)

    L_inv = L_inv @ P.T
    # Step 2: Recover L by inverting L_inv via transposition of gauss_elimination result
    # Note: We can use gauss_elimination to "invert" L_inv:
    _, _, L = gauss_elimination(L_inv, np.eye(n), pivoting=False)

    return P.T, L, U

You can always check that your LU decomposition works as

$LU - A = O$ (a zero matrix)

In [18]:
answer['simple_PLU'] = [
    find_plu(input['A'], pivoting=False) for input in inputs['simple_cases']
]

answer['pivoting_PLU'] = [
    find_plu(input['A'], pivoting=True) for input in inputs['pivoting_cases']
]

answer['nullspace_PLU'] = [
    find_plu(input['A'], pivoting=True) for input in inputs['nullspace_cases']
]


# This should be close to zero
print("=== SIMPLE CASES ===")
for index in range(len(inputs['simple_cases'])):
    print(np.abs(answer['simple_PLU'][index][0] @ answer['simple_PLU'][index][1] @ answer['simple_PLU'][index][2] - inputs['simple_cases'][index]['A']).sum())

print("=== PIVOTING CASES ===")
for index in range(len(inputs['pivoting_cases'])):
    print(np.abs(answer['pivoting_PLU'][index][0] @ answer['pivoting_PLU'][index][1] @ answer['pivoting_PLU'][index][2] - inputs['pivoting_cases'][index]['A']).sum())

print("=== NULLSPACE CASES ===")
for index in range(len(inputs['nullspace_cases'])):
    print(np.abs(answer['nullspace_PLU'][index][0] @ answer['nullspace_PLU'][index][1] @ answer['nullspace_PLU'][index][2] - inputs['nullspace_cases'][index]['A']).sum())

=== SIMPLE CASES ===
1.9917401061775308e-13
1.0749179324420766e-12
2.4424906541753444e-15
6.652086289212396e-14
1.7763568394002505e-15
3.1086244689504383e-15
1.1324274851176597e-14
0.0
4.956987201662079e-14
2.5091040356528543e-13
0.0
6.62433071359677e-15
2.573597456600323e-13
6.608126744213547e-14
1.6625589793761757e-14
0.0
0.0
1.7763568394002505e-15
0.0
0.0
3.192155534612712e-14
0.0
0.0
2.34035013590983e-13
0.0
0.0
0.0
1.4697397339573243e-13
1.4193913387116043e-13
0.0
=== PIVOTING CASES ===
3.437609987124992e-14
3.8351994838750226e-14
1.1773650837334399e-14
4.711299144558125e-14
0.0
6.661338147750939e-15
4.425250762176365e-14
2.1354163858423458e-14
3.2791211563518314e-14
1.1168580240690859e-13
8.087974734394265e-15
0.0
1.5654144647214707e-14
2.882445379224273e-14
3.9968028886505635e-15
8.724052348695988e-14
7.489044466704765e-14
0.0
5.786076127080058e-14
1.2325035337565058e-13
3.137176579371335e-14
7.69546187301989e-14
3.231947892664425e-14
0.0
7.93809462606987e-16
0.0
6.1216563462994

# Finding NullSpace

Now find null-space of the matrix using Gauss Elimination. To do so, analyze the matrix $U$ that appears after Gauss Elimination. Observing this matrix
it is easy to write out the vectors of Null Space.

Note that variable `eps` stands for tolerance. If a value deviates from 0 more than on `eps`, we consider it to be different from `0`.

In [19]:
def find_null_space(A, pivoting=True, eps=1e-12):
    n, m = A.shape
    _, U, _ = gauss_elimination(A, np.zeros((n, 1)), pivoting=pivoting)
    pivots = []
    for i in range(min(n, m)):
        if np.abs(U[i, i]) > eps:
            pivots.append(i)
    free_vars = [j for j in range(m) if j not in pivots]
    if not free_vars:
        return np.zeros((m, 0))
    Z = []
    for free in free_vars:
        z = np.zeros(m)
        z[free] = 1
        for i in reversed(range(len(pivots))):
            row = pivots[i]
            z[row] = -np.sum(U[row, free_vars] * z[free_vars])
        Z.append(z)
    return np.stack(Z, axis=1)

In [20]:
answer['nullspace'] = [
    find_null_space(input['A'], pivoting=True) for input in inputs['nullspace_cases']
]

for index in range(len(answer['nullspace'])):
    print(inputs['nullspace_cases'][index]['A'] @ answer['nullspace'][index])

[[ 0.  0.  6.  0. -8.  0.]
 [ 0.  1.  0.  0. -3. -2.]
 [ 0. -3.  6. -1.  1.  7.]
 [ 0. -1. -3.  1.  7.  1.]
 [ 0. -2. -3. -5. 10. 10.]
 [ 0. -4.  6. -2.  4. 11.]]
[[0.]
 [0.]]
[[ 0.  1.  2.]
 [ 0.  1. -3.]
 [ 0. -4. -3.]]
[[ 0.  5. -7. -2.  5.]
 [ 0.  0.  9. -7. -2.]
 [ 0. -3.  5. -4. 14.]
 [ 0.  3. -2. -1. -5.]
 [ 0.  2.  1. -3. -3.]]
[[  0.   4.   2.  -1.  -2.]
 [  0.   2.   0.  -2.  -3.]
 [  0.   0.  -2.   3. -10.]
 [  0.  -2.   2.   1.   3.]
 [  0.   0.  -2.   0.  -3.]]
[[ 0.         -0.33333333 -1.         -0.66666667 -1.          0.
   0.33333333]
 [ 0.          6.22222222 13.33333333  5.44444444 10.         -7.66666667
   0.11111111]
 [ 0.          3.66666667  5.         -5.66666667 -7.         -2.
   1.33333333]
 [ 0.          0.44444444 -5.33333333 -1.11111111 -6.         -3.33333333
  -1.77777778]
 [ 0.          0.          0.          0.          0.          0.
   0.        ]
 [ 0.          3.22222222 14.33333333  2.44444444 11.         -1.66666667
   4.11111111]
 [ 0.      

# QR decomposition

Now again we will consider matrix $A^T$ in context of SLE:

$A^T \mathbf x = I \mathbf b$

Out of matrix $A$ we can make an orthonormal matrix $Q$ by orthonoramlizing the rows, this process will turn the $I$ matrix on the right to lower-triangular matrix $L^*$:

$A^T \mathbf x = I \mathbf b \rightarrow Q^T \mathbf x = L^* \mathbf b$

After that we should invert matrix $L^*$ and we will get:

$I Q^T \mathbf x = L^* \mathbf b \rightarrow L Q^T \mathbf x = \mathbf b$

If we transpose that matrix, we will get  QR decomposition of matrix $A$:

$A = Q R,$

where $R$ is upper-triangular matrix.

The task is to:
1. Implement Gramm-Schmidt orthonormalization procedure
2. Combining Gramm-Schmidt algorithm and Gaussian Algorithm, implement QR decomposition

# Task 5: Gramm-Schmidt Algorithm for Matrices

Implement Gramm-Schmidt process for **rows** of matrix $A$ (and do the same transforms to matrix $B$)

In [21]:
def gramm_schmidt(A, B):
    A = A.copy().astype(float)
    B = B.copy().astype(float)
    ## YOUR CODE HERE
    # --placeholder start--
    for i in range(A.shape[1]):
        for j in range(i):
            factor = np.dot(A[i, :], A[j, :])
            A[i, :] -= factor * A[j, :]
            B[i, :] -= factor * B[j, :]
        factor = np.linalg.norm(A[i, :])
        A[i, :] /= factor
        B[i, :] /= factor
    # --placeholder end--
    return A, B

In [22]:
answer['gramm_schmidt_simple_cases'] = [
    gramm_schmidt(input['A'], np.eye(input['A'].shape[0])) for input in inputs['simple_cases']
]

answer['gramm_schmidt_pivoting_cases'] = [
    gramm_schmidt(input['A'], np.eye(input['A'].shape[0])) for input in inputs['pivoting_cases']
]

# answer['gramm_schmidt_nullspace_cases'] = [
    # gramm_schmidt(input['A'], np.eye(input['A'].shape[0])) for input in inputs['nullspace_cases']
# ]

print("===== Should be close to 1s: =====")
print((answer['gramm_schmidt_simple_cases'][0][0] ** 2).sum(axis=1))
print()
print("===== Should be close to L: =====")
print(answer['gramm_schmidt_simple_cases'][0][1])

===== Should be close to 1s: =====
[1. 1. 1. 1. 1. 1.]

===== Should be close to L: =====
[[ 1.41421356e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 3.53553391e-02  5.89255651e-02  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-7.15093487e-01  1.46213273e+00  4.42325868e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.03569033e+00  3.92181405e+00  1.02602389e+01  6.35223403e-01
   0.00000000e+00  0.00000000e+00]
 [ 1.26738917e+00 -9.18397948e-01 -3.39807241e+00  3.67359179e-01
   1.04697366e+00  0.00000000e+00]
 [ 5.29513565e+00  4.98134983e+01  1.26691023e+02  1.62776392e+01
   4.80484531e+01  5.09901951e+00]]


# Task 6: QR decomposition

Implement QR decomposition algorithm. Note that we work on rows on matrix $A$ with our Gauss and Gramm-Schmidt algorithms, but QR decomposition is defined for column matrix 

In [23]:
def find_qr(A):
    B = np.eye(A.T.shape[0])
    ## YOUR CODE HERE
    # --placeholder start--
    QT, L1 = gramm_schmidt(A.T, B)
    _, I, LT = gauss_elimination(L1, np.eye(A.shape[0]))
    Q = QT.T
    R = LT.T
    # --placeholder end--
    return Q, R

In [24]:
answer['QR_simple_cases'] = [
    find_qr(input['A']) for input in inputs['simple_cases']
]

print("===== Should be close to O: =====")
print(answer['QR_simple_cases'][0][0] @ answer['QR_simple_cases'][0][1] - inputs['simple_cases'][0]['A'])

===== Should be close to O: =====
[[ 0.00000000e+00 -1.26075241e-17  0.00000000e+00  0.00000000e+00
  -3.33066907e-16 -3.21964677e-15]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.77635684e-15]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.22044605e-16
  -4.44089210e-16 -1.33226763e-15]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  6.30376205e-18  0.00000000e+00  0.00000000e+00
   0.00000000e+00  4.44089210e-16]
 [ 0.00000000e+00 -8.88178420e-16 -8.88178420e-16  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


In [25]:
json_tricks.dump(answer, '.answer.json')

'{"gauss_elimination_simple_cases": [[{"__ndarray__": [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]], "dtype": "float64", "shape": [6, 6], "Corder": true}, {"__ndarray__": [[1.0, -0.0, -1.0, 1.0, -0.25, -0.25], [-0.0, 1.0, 0.5, 1.5, -1.1666666666666667, 1.8333333333333333], [-0.0, -0.0, -0.0, -0.0, 1.0, 1.0], [0.0, 0.0, -0.5, 1.5, 0.0, 1.0], [0.0, 0.0, 0.0, 2.0, 0.0, 0.0], [0.0, 0.0, 0.5, -19.5, 0.0, 0.0]], "dtype": "float64", "shape": [6, 6], "Corder": true}, {"__ndarray__": [-16.0, -4.333333333333333, 8.00000000000001, -10.00000000000001, -14.000000000000016, 139.0000000000001], "dtype": "float64", "shape": [6]}], [{"__ndarray__": [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1